In [1]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q datasets
!pip install -q evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [2]:
cp -r /kaggle/input/deep-seek-thanh/deep-seek-Thanh /kaggle/working/


In [3]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load mô hình gốc
base_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct")

# Load mô hình fine-tuned
model = PeftModel.from_pretrained(base_model, "/kaggle/working/deep-seek-Thanh")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct")


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 2048)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
               

In [6]:
from datasets import load_dataset

# Tải bộ dữ liệu từ Hugging Face
dataset = load_dataset("gretelai/synthetic_text_to_sql")

# Truy cập phần train và test
train_data = dataset['train']
test_data = dataset['test']

# In 2 mẫu đại diện dưới dạng JSON từ phần train và test
import json

# Lấy 2 mẫu đầu tiên từ phần train và test
train_sample = train_data[0]
test_sample = test_data[0]

# In dưới dạng JSON
print("Train Sample:", json.dumps(train_sample, indent=2))
print("Test Sample:", json.dumps(test_sample, indent=2))


README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Train Sample: {
  "id": 5097,
  "domain": "forestry",
  "domain_description": "Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.",
  "sql_complexity": "single join",
  "sql_complexity_description": "only one join (specify inner, outer, cross)",
  "sql_task_type": "analytics and reporting",
  "sql_task_type_description": "generating reports, dashboards, and analytical insights",
  "sql_prompt": "What is the total volume of timber sold by each salesperson, sorted by salesperson?",
  "sql_context": "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180

In [7]:
LEFT_INDEX = 1000
RIGHT_INDEX = 5851
BATCH_SIZE = 16

In [8]:
references = [
    test_data[idx]['sql'] for idx in range(LEFT_INDEX, RIGHT_INDEX)
]

In [9]:
BASE_TEXT2SQL_PROMPT = """
# Follow these instruction:
You will be given context of making tables of a database. Your job is to write correct
error free SQL query based on the question asked. Please make sure:

1. Do not add ``` at start / end of the query. It should be a single line query in a  single line (string format)
2. Make sure the column names are correct and exists in the table
3. For column names which has a space with it, make sure you have put `` in that column name
4. Think step by step and always check context and question and the column names before writing the
query. 

# Hre is context for the SQL query:
{additional_knowledge}

# Question: {question}

# SQL: 
"""

few_shot_examples = '''
single join: only one join (specify inner, outer, cross):
  Sample 1:
    SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?
    SQL Query: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;
  Sample 2:
    SQL Prompt: What is the average water temperature for each fish species in February?
    SQL Query: SELECT SpeciesName, AVG(WaterTemp) as AvgTemp FROM SpeciesWaterTemp INNER JOIN FishSpecies ON SpeciesWaterTemp.SpeciesID = FishSpecies.SpeciesID WHERE MONTH(Date) = 2 GROUP BY SpeciesName;
  Sample 3:
    SQL Prompt: What is the average income of farmers in each district in India?
    SQL Query: SELECT d.name, AVG(f.income) FROM farmers_india f JOIN districts_india d ON f.district_id = d.id GROUP BY d.name;

aggregation: aggregation functions (COUNT, SUM, AVG, MIN, MAX, etc.), and HAVING clause:
  Sample 1:
    SQL Prompt: List all the unique equipment types and their corresponding total maintenance frequency from the equipment_maintenance table.
    SQL Query: SELECT equipment_type, SUM(maintenance_frequency) AS total_maintenance_frequency FROM equipment_maintenance GROUP BY equipment_type;
  Sample 2:
    SQL Prompt: What is the total trade value and average price for each trader and stock in the trade_history table?
    SQL Query: SELECT trader_id, stock, SUM(price * quantity) as total_trade_value, AVG(price) as avg_price FROM trade_history GROUP BY trader_id, stock;
  Sample 3:
    SQL Prompt: Show the conservation status of marine mammals and birds by year.
    SQL Query: SELECT animal_group, conservation_status, year, COUNT(*) FROM combined GROUP BY animal_group, conservation_status, year;

basic SQL: basic SQL with a simple select statement:
  Sample 1:
    SQL Prompt: How many marine species are found in the Southern Ocean?
    SQL Query: SELECT COUNT(*) FROM marine_species WHERE location = 'Southern Ocean';
  Sample 2:
    SQL Prompt: What is the total spending on humanitarian assistance by the European Union in the last 3 years?
    SQL Query: SELECT SUM(spending) FROM defense.eu_humanitarian_assistance WHERE year BETWEEN 2019 AND 2021;
  Sample 3:
    SQL Prompt: Delete a program's outcome data
    SQL Query: DELETE FROM Program_Outcomes WHERE program_id = 1002;

window functions: window functions (e.g., ROW_NUMBER, LEAD, LAG, RANk, NTILE, PERCENT_RANK, etc.) with partitioning and ordering:
  Sample 1:
    SQL Prompt: Find the energy efficiency upgrades with the highest cost and their types.
    SQL Query: SELECT type, cost FROM (SELECT type, cost, ROW_NUMBER() OVER (ORDER BY cost DESC) as rn FROM upgrades) sub WHERE rn = 1;
  Sample 2:
    SQL Prompt: Which autonomous vehicles have the highest safety ratings in the 'auto_show' table?
    SQL Query: SELECT vehicle_name, safety_rating FROM (SELECT vehicle_name, safety_rating, RANK() OVER (ORDER BY safety_rating DESC) as safety_rank FROM auto_show WHERE vehicle_name LIKE '%Autonomous%') AS auton_ranks WHERE safety_rank = 1;
  Sample 3:
    SQL Prompt: Find the title and genre of the top 3 movies with the highest ratings from studios based in Italy, ordered by ratings in descending order.
    SQL Query: SELECT title, genre FROM (SELECT title, genre, studio, rating, ROW_NUMBER() OVER (PARTITION BY studio ORDER BY rating DESC) as rank FROM movies WHERE studio LIKE '%Italy%') subquery WHERE rank <= 3 ORDER BY rating DESC;

subqueries: subqueries, including correlated and nested subqueries:
  Sample 1:
    SQL Prompt: What is the average sea surface temperature in the Pacific Ocean per year?
    SQL Query: SELECT AVG(temperature) FROM (SELECT temperature FROM Ocean WHERE region = 'Pacific' GROUP BY temperature, YEAR(STR_TO_DATE(region, '%Y'))) AS PacificAverage;
  Sample 2:
    SQL Prompt: What are the common cybersecurity policies across all departments, according to our Policy Analysis database?
    SQL Query: SELECT policy FROM PolicyAnalysis GROUP BY policy HAVING COUNT(DISTINCT department) = (SELECT COUNT(DISTINCT department) FROM PolicyAnalysis);
  Sample 3:
    SQL Prompt: What is the total number of space missions led by women?
    SQL Query: SELECT COUNT(*) FROM space_missions WHERE leader IN (SELECT name FROM astronauts WHERE gender = 'Female');

multiple_joins: two or more joins (specify inner, outer, cross):
  Sample 1:
    SQL Prompt: What are the names and habitats of all animals that are part of an education program in the 'Americas' region?
    SQL Query: SELECT a.name, h.name FROM Animals a INNER JOIN AnimalEducation ae ON a.id = ae.animal_id INNER JOIN EducationPrograms e ON ae.program_id = e.id INNER JOIN Coordinators c ON e.coordinator_id = c.id INNER JOIN Habitat h ON a.habitat_id = h.id WHERE c.region = 'Americas';
  Sample 2:
    SQL Prompt: What is the total biomass of deciduous trees per wildlife habitat?
    SQL Query: SELECT h.name habitat, SUM(t.biomass) total_biomass FROM trees t JOIN tree_habitat_associations tha ON t.id = tha.tree_id JOIN wildlife_habitats h ON tha.habitat_id = h.id JOIN tree_types tt ON t.tree_type_id = tt.id WHERE tt.name = 'Deciduous' GROUP BY h.name;
  Sample 3:
    SQL Prompt: Get the name and country of the top 3 customers for organic cosmetics
    SQL Query: SELECT customers.name, customers.country FROM customers JOIN purchases ON customers.id = purchases.customer_id JOIN products ON purchases.product_id = products.id WHERE products.organic = TRUE GROUP BY customers.id ORDER BY SUM(purchases.quantity) DESC LIMIT 3;

set operations: set operations such as UNION, INTERSECT, and EXCEPT:
  Sample 1:
    SQL Prompt: Which menu items are served at both Location1 and Location2, but not at Location3?
    SQL Query: SELECT menu_item FROM menu_items WHERE location = 'Location1' INTERSECT SELECT menu_item FROM menu_items WHERE location = 'Location2' EXCEPT SELECT menu_item FROM menu_items WHERE location = 'Location3';
  Sample 2:
    SQL Prompt: How many players from Africa and South America have adopted VR technology?
    SQL Query: SELECT 'Africa' AS continent, COUNT(DISTINCT p.player_id) AS num_players FROM players p JOIN vr_tech v ON p.player_id = v.player_id WHERE p.region = 'Africa' UNION ALL SELECT 'South America' AS continent, COUNT(DISTINCT p.player_id) AS num_players FROM players p JOIN vr_tech v ON p.player_id = v.player_id WHERE p.region = 'South America';
  Sample 3:
    SQL Prompt: What are the total number of works in the 'Artworks' and 'Exhibitions' tables?
    SQL Query: SELECT COUNT(*) FROM Artworks UNION ALL SELECT COUNT(*) FROM Exhibitions;

CTEs: common table expressions:
  Sample 1:
    SQL Prompt: Add a new spacecraft to the Spacecrafts table.
    SQL Query: WITH new_spacecraft AS (VALUES (1, 'New Horizons 2', 'NASA', '2025-01-01', 'Active')) INSERT INTO Spacecrafts (Sid, Name, Manufacturer, Launch_Date, Status) SELECT * FROM new_spacecraft;
  Sample 2:
    SQL Prompt: Delete a publication record from the "publications" table
    SQL Query: WITH deleted_publication AS (DELETE FROM publications WHERE id = 1 RETURNING *) SELECT * FROM deleted_publication;
  Sample 3:
    SQL Prompt: Delete an artwork from the 'Impressionist Art' exhibition
    SQL Query: WITH artwork_to_delete AS (DELETE FROM exhibition_artworks WHERE exhibition_id = 1 AND artwork_id = 1) SELECT * FROM artwork_to_delete;'''


In [10]:
import re
import json
import torch
from time import time
from torch.utils.data import DataLoader

extracted_sqls_with_context = []
# Định nghĩa dataset từ test_data
class SQLDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, left, right):
        self.dataset = dataset.select(range(left, right)).to_dict()
        self.samples = [dict(zip(self.dataset.keys(), values)) for values in zip(*self.dataset.values())]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return BASE_TEXT2SQL_PROMPT.format(
            question=sample['sql_prompt'],
            additional_knowledge=sample['sql_context'],
            few_shot_examples=few_shot_examples
        )

# Bắt đầu đo thời gian
start_time = time()

# Tạo DataLoader
sql_dataset = SQLDataset(test_data, LEFT_INDEX, RIGHT_INDEX)
dataloader = DataLoader(sql_dataset, batch_size=BATCH_SIZE, shuffle=False)



In [ ]:
# Định nghĩa file output
filename = f'Prem-1B-SQL_{LEFT_INDEX}_{RIGHT_INDEX}.jsonl'

total_samples = 0  # Đếm tổng số mẫu

with open(filename, 'w') as json_file:
    for batch_idx, batch_sql_prompts in enumerate(dataloader):
        batch_start_time = time()  # Đo thời gian cho từng batch

        tokenizer.padding_side = "left"
        inputs = tokenizer(batch_sql_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

        # Sinh output từ mô hình
        with torch.no_grad():
          generated_ids = model.generate(
              **inputs,
              max_new_tokens=512,
              pad_token_id=tokenizer.eos_token_id
          )


        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
        ]
        # Giải mã output
        generated_sqls = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)


        for idx, generated_sql in enumerate(generated_sqls):
            generated_sql = generated_sql.strip()  # Loại bỏ khoảng trắng hai đầu
            generated_sql = re.sub(r"^[^\w]+", "", generated_sql)   # Loại bỏ các ký tự không hợp lệ bên trái
            generated_sql = generated_sql.split(';', 1)[0] + ';' if ';' in generated_sql else generated_sql  # Lấy đến dấu ';' đầu tiên
            total_samples += 1
            extracted_sqls_with_context.append(generated_sql)
            result = {'id': LEFT_INDEX + batch_idx * BATCH_SIZE + idx + 1, 'answer': generated_sql}
            json.dump(result, json_file)
            json_file.write("\n")




        batch_end_time = time()
        print(f"⏱️ Batch {batch_idx + 1} processed in {batch_end_time - batch_start_time:.2f} seconds.")

# Kết thúc đo thời gian
end_time = time()
elapsed_time = end_time - start_time

# In ra kết quả
print(f"✅ Data saved to {filename}")
print(f"⏳ Total time taken: {elapsed_time:.2f} seconds")
if total_samples > 0:
    print(f"⏱️ Average time per sample: {elapsed_time / total_samples:.4f} seconds")


⏱️ Batch 1 processed in 14.04 seconds.
⏱️ Batch 2 processed in 9.02 seconds.
⏱️ Batch 3 processed in 15.92 seconds.
⏱️ Batch 4 processed in 10.68 seconds.
⏱️ Batch 5 processed in 9.31 seconds.
⏱️ Batch 6 processed in 13.53 seconds.
⏱️ Batch 7 processed in 12.16 seconds.
⏱️ Batch 8 processed in 14.56 seconds.
⏱️ Batch 9 processed in 11.88 seconds.
⏱️ Batch 10 processed in 15.12 seconds.
⏱️ Batch 11 processed in 10.90 seconds.
⏱️ Batch 12 processed in 19.68 seconds.
⏱️ Batch 13 processed in 10.77 seconds.
⏱️ Batch 14 processed in 9.14 seconds.
⏱️ Batch 15 processed in 14.76 seconds.
⏱️ Batch 16 processed in 12.67 seconds.
⏱️ Batch 17 processed in 12.67 seconds.
⏱️ Batch 18 processed in 17.23 seconds.
⏱️ Batch 19 processed in 13.19 seconds.
⏱️ Batch 20 processed in 17.54 seconds.
⏱️ Batch 21 processed in 11.24 seconds.
⏱️ Batch 22 processed in 12.58 seconds.
⏱️ Batch 23 processed in 14.27 seconds.
⏱️ Batch 24 processed in 10.87 seconds.
⏱️ Batch 25 processed in 14.90 seconds.
⏱️ Batch 26 

In [ ]:
import evaluate

In [ ]:
# Tính BLEU
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=extracted_sqls_with_context, references=references)

print(results)
